In [1]:
import graphlab
import numpy as np
import pandas as pd
# import matplotlib 

In [2]:
# Drop the duplicates 
Total_Test = pd.read_csv("../Total_Test_AfterProcessing.csv")
Total_Test = Total_Test.drop_duplicates('answer_id')
Total_Test = Total_Test.fillna(0)
Total_Test['question_id'] = Total_Test['question_id'].astype(str)
Total_Test['answer_voted'] = Total_Test['answer_voted'].astype(float)
# Total_Test # 42512 rows × 18 columns

In [3]:
Total_Test.to_csv('./Total_Test.csv', encoding='utf-8', index = False)
Total_Test= pd.read_csv('./Total_Test.csv')
# Total_Test  # 42512 rows × 18 columns

In [4]:
# Total_Test  # 42512 rows × 18 columns
been_voted = Total_Test[ Total_Test['answer_voted'] > 0 ]
un_voted = Total_Test[ Total_Test['answer_voted'] == 0 ]
been_voted = been_voted.drop_duplicates('question_id')
been_voted.to_csv('./Been_Voted.csv', encoding='utf-8', index = False)
# been_voted  # 17039 rows × 18 columns
# un_voted    # 14823 rows × 18 columns

In [5]:
been_voted = pd.read_csv('./Been_Voted.csv')
been_voted_qid = been_voted['question_id']
# len(been_voted_qid)   # 17039
# type(been_voted_qid)  # Series
# been_voted_qid[10]

In [6]:
# been_voted_qid

In [7]:
# Our Strategy is Select Qid in Voted Question List, 
# Select Similar Question From Total Test Set 

In [8]:
question_content = Total_Test[['question_id','question_content','question_tags']]
# question_content  # 42512 rows × 3 columns
question_content = question_content.drop_duplicates('question_id')
# question_content # 31862 rows × 3 columns
question_content.to_csv('./Question_Content.csv', encoding='utf-8', index = False)

In [9]:
question_content = graphlab.SFrame.read_csv('./Question_Content.csv')
# question_content  # 31862 rows x 3 columns

2016-04-28 15:49:44,014 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.8.5 started. Logging: /tmp/graphlab_server_1461829782.log


Finished parsing file /home/zpgao/ML/Best_Answer/SVM_RANK_master/SIMILAR/Question_Content.csv

Parsing completed. Parsed 100 lines in 0.139422 secs.

This non-commercial license of GraphLab Create is assigned to beyond_acm@163.com and will expire on January 04, 2017. For commercial licensing options, visit https://dato.com/buy/.
------------------------------------------------------

Finished parsing file /home/zpgao/ML/Best_Answer/SVM_RANK_master/SIMILAR/Question_Content.csv

Parsing completed. Parsed 31862 lines in 0.144323 secs.


Inferred types from first line of file as 
column_type_hints=[int,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


In [10]:
# add 'text' && 'word_count' && 'tfidf' features
question_content['text'] = question_content['question_tags'] + question_content['question_content']
question_content['word_count'] = graphlab.text_analytics.count_words(question_content['text'])

In [11]:
tfidf = graphlab.text_analytics.tf_idf(question_content['word_count'])

In [12]:
question_content['tfidf'] = tfidf

In [13]:
# been_voted_qid
type ( been_voted_qid[99] )

numpy.int64

In [14]:
# question_content[ question_content['question_id'] == been_voted_qid[5] ]

In [15]:
# question_content
# len(been_voted)  17039

In [16]:
def Select_Question( num_question, init_qid, step_size = 10):
    # Initial selected_question
    selected_question = question_content[ question_content['question_id'] == been_voted_qid[init_qid] ]
    for i in range(num_question) :
        # Random Select Index 
        selected_qid = been_voted_qid[init_qid + i*16 + 77]
        # print init_qid + i*16 + 77
        selected_content = question_content[ question_content['question_id'] == selected_qid ]
        selected_question = selected_question.append(selected_content)
    print init_qid + i*16 + 77
    return selected_question

In [17]:
# Random Select 50 voted / Unvoted Question
selected_00 = Select_Question(99, 7)
selected_01 = Select_Question(99, 1700)
selected_02 = Select_Question(99, 3350)
# selected_03 = Select_Question(99, 5000)
# selected_04 = Select_Question(99, 6650)
# selected_05 = Select_Question(99, 8300)
# selected_06 = Select_Question(99, 9950)
# selected_07 = Select_Question(99, 11600)
# selected_08 = Select_Question(99, 13250)
# selected_09 = Select_Question(99, 14900) 

1652
3345
4995


In [18]:
# len( selected )
# len( selected_02 )
# type( selected_01 )

In [19]:
knn_model = graphlab.nearest_neighbors.create(question_content, features=['tfidf'], label='question_id')

Starting brute force nearest neighbors model training.

In [21]:
# import json
# import pickle

def writeDict(dict, filename, sep):
    with open(filename, "w") as f:
#        for key in dict.keys():            
#            f.write(str(key) + ":" + str(dict[key]) + "\n")
        for key, value in sorted( dict.items() ):
            f.write( str(key) + sep + ','.join(str(x) for x in value) + '\n' )
            
def readDict(filename, sep):
    with open(filename, "r") as f:
        dict = {}
        for line in f:
            values = line.split(sep)
            key = values[0]
            value = values[1].split(',')
            #dict[values[0]] = {int(x) for x in values[1:len(values)]}
            dict[key] = value
        return(dict)

In [22]:
# selected_qid is a list 
# k is the num of similar questions
def Get_Similar_SelectedID(selected_qid, similar_num=5) :
    Similar_Qid_List = {}
    for i in range( len(selected_qid) ) :
        key = 'Q' + str(i) + '_Similar_List'
        Q_i = selected_qid[i:i+1]
        value = knn_model.query(Q_i, k = similar_num)['reference_label']
        Similar_Qid_List[key] = value
    return Similar_Qid_List

In [23]:
similar_qid_00 = Get_Similar_SelectedID(selected_00)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00313853  | 7.228ms      |

...

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00313853  | 14.339ms     |

| Done         |         | 100         | 86.216ms     |

+--------------+---------+-------------+--------------+

In [24]:
similar_qid_01 = Get_Similar_SelectedID(selected_01)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00313853  | 8.171ms      |

...

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00313853  | 5.822ms      |

| Done         |         | 100         | 85ms         |

+--------------+---------+-------------+--------------+

In [25]:
similar_qid_02 = Get_Similar_SelectedID(selected_02)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00313853  | 6.722ms      |

...

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00313853  | 27.702ms     |

| Done         |         | 100         | 123.043ms    |

+--------------+---------+-------------+--------------+

In [26]:
def Get_Index_QuestionID(qid) :
    return Total_Test[ Total_Test['question_id'] == qid].index.values

In [27]:
def Get_IndexList(Q_List) :
    IndexList = []
    for qid in Q_List:
        indexes = Get_Index_QuestionID( qid )
        if len(indexes) == 0:
            indexes = Get_Index_QuestionID( qid )
        # print qid, indexes
        for index in indexes:
            IndexList.append(index)
            # print len( IndexList )
    return IndexList

In [30]:
# From Similar Qid List Found Similar Index
def Get_Index_SimilarQID( Similar_Qid_List ) :
    Similar_Index_List = {}
    for i in range( len(Similar_Qid_List) ) :
        key = 'Q' + str(i) + '_Similar_List'
        # value is a list of similar qid
        Qid_List = Similar_Qid_List[key]
        Index_List = Get_IndexList(Qid_List)
        Similar_Index_List[key]  = Index_List
    
    return Similar_Index_List

In [31]:
similar_index_00 = Get_Index_SimilarQID(similar_qid_00)

In [32]:
similar_index_01 = Get_Index_SimilarQID(similar_qid_01)

In [33]:
similar_index_02 = Get_Index_SimilarQID(similar_qid_02)

In [34]:
writeDict(similar_qid_00, './1000x5/SIMILAR_QID/similar_qid_00.txt', ':')
writeDict(similar_index_00, './1000x5/SIMILAR_INDEX/similar_index_00.txt', ':')

writeDict(similar_qid_01, './1000x5/SIMILAR_QID/similar_qid_01.txt', ':')
writeDict(similar_index_01, './1000x5/SIMILAR_INDEX/similar_index_01.txt', ':')

writeDict(similar_qid_02, './1000x5/SIMILAR_QID/similar_qid_02.txt', ':')
writeDict(similar_index_02, './1000x5/SIMILAR_INDEX/similar_index_02.txt', ':')

In [36]:
test = readDict('./1000x5/SIMILAR_QID/similar_qid_00.txt', ':')

In [37]:
test == similar_qid_00

True